# Use Case 2: Air Quality Monitoring 🌬️

*The provided notebook is inspired by Open Sourced City Scanner, a project developed at Senseable City Lab. More information about this project can be found [here](https://github.com/MIT-Senseable-City-Lab/OSCS/tree/main/Learn/Coding%20Exercise).*

This notebook will guide you through how to analyze air quality data on a map from the octopus.

## Initial Setup

In the beginning of this setup, you will connect to your google drive and download libraries needed for analysis. Then, you will need to clean your data to make it ready to plot in a map.

In [ ]:
#Block 1
#this block is going to allow the notebook to connect to your google drive so you can interact with it
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Block 2
#here, we install the tool we are going to use to make some maps, called Folium
%%capture
!pip install folium

In [ ]:
#Block 3
#this block of code is where we get the infrastructure for the notebook set up, by calling libraries
import csv
import numpy as np

#these libraries will help us read in and format the data correctly
import pytz
import time
import pandas as pd
from datetime import datetime
import os

#these libraries will help us with our time series analysis
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
import matplotlib.dates as mdates

#these libraries will support the mapping work
import folium
from folium import plugins
import branca.colormap as cm
from matplotlib.dates import DateFormatter
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [ ]:
#Block 4
#adjust the path below to match your google drive setup! please use the format below:
os.chdir('/content/drive/My Drive/path to correct folder/')

In [ ]:
# Block 5
# Read the raw data from a CSV file into a DataFrame
# adjust name to correct file
filename = "octopus_data_test.txt"
df = pd.read_csv(filename, names=["timestamp", "temperature", "humidity", "pressure", "pm1", "pm4", 'pm25', "pm10"])

# Display the first values in the dataframe
df.head()

# Display the last values in the dataframe
df.tail()

# Part 1 - Data Preprocessing
Depending on you use case and what you want to answer with your data, different pre-processing steps should be done to get the best results possible. In this case, we will conduct Handling Missing Values, Outlier Detection and Removal, Resampling, Normalization, Smoothing and Seasonal Decomposition.

In [ ]:
# Block 5
# Handling missing values
# Handling missing values for multiple columns using forward fill
columns_to_fill = ["temperature", "humidity", "pressure", "pm1", "pm4", "pm25", "pm10"]
for column in columns_to_fill:
    df[column].fillna(method='ffill', inplace=True)


In [ ]:
# Block 6
# Outlier detection and removal (using Z-score)
columns_to_check = ["temperature", "humidity", "pressure", "pm1", "pm4", "pm25", "pm10"]
z_threshold = 3

for column in columns_to_check:
    df[column + '_z'] = (df[column] - df[column].mean()) / df[column].std()

condition = np.logical_and.reduce([(df[col + '_z'].abs() < z_threshold) for col in columns_to_check])
df = df[condition]

# Optionally, drop the intermediate Z-score columns if they are no longer needed
df.drop([col + '_z' for col in columns_to_check], axis=1, inplace=True)

In [ ]:
# Block 7
# Resampling
# This part will depend on the frequency of your collected data, and what you want to analyse.
# In this example, the data is collected every second, but we want to analyze the mean values every 15 min
# '15T' specifies the new frequency, indicating every 15 minutes. You can change to '1H' if you want every hour.
# .mean() specifies the aggregation method, meaning that within each new 15-minute interval, the mean of the values will be calculated.

df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)
df = df.resample('15T').mean()

In [ ]:
# Block 8
# Normalization
df_normalized = (df - df.mean()) / df.std()

In [ ]:
# Block 9
# Smoothing (using 5-point moving average)
df_smoothed = df.rolling(window=5).mean()

In [ ]:
# Block 10
# Seasonal decomposition
decomposition = seasonal_decompose(df["temperature"], period=24)  # Assuming daily seasonality 24h
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Part 2 - Time Series

As this version of the octopus also collects temperature and humidity, lets have a look at the data!

In [ ]:
# Block 11
# Plot Temperature

plt.figure(figsize=(12, 6))
plt.plot(df.index, df['temperature'], label='Temperature', color='red', marker='o')
plt.xlabel('Index')
plt.ylabel('°C')
plt.title('First 10 Rows of Environmental Data')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Block 12
# Plot Humidity

plt.figure(figsize=(12, 6))
plt.plot(df.index, df['humidity'], label='Humidity', color='blue', marker='o')
plt.xlabel('Index')
plt.ylabel('%')
plt.title('First 10 Rows of Environmental Data')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Block 12
# Plot Barometric Pressure

plt.figure(figsize=(12, 6))
plt.plot(df.index, df['pressure'], label='Pressure', color='blue', marker='o')
plt.xlabel('Index')
plt.ylabel('%')
plt.title('First 10 Rows of Environmental Data')
plt.legend()
plt.grid(True)
plt.show()

# Part 3 - Mapping + Hotspot Analysis

Now that we created some time series graphs, we will work on creating maps. Let's start with a PM2.5 map!

In [ ]:
#Block 12
#we're going to make a map. we start by setting a center point for the map to display the data
coords = pmdata.loc[:,['latitude','longitude']].values #lat and lon are collected from CityScanner GPS
start_point=coords[0]

In [ ]:
#Block 13
#here, we set up the specifications for the map
newmap = folium.Map(location= start_point, tiles='Stamen Terrain', zoom_start=14)
colormap = cm.LinearColormap(colors=['blue', 'green', 'yellow'], vmin=0, vmax=21)
colormap.caption = 'PM 2.5 (ug/m^3)' #change this to represent the variable of interest!
colormap.add_to(newmap)


#this will loop through the data and show us where it's coming from
#each point on the map will display the PM2.5 value from that spot, in micrograms per cubic meter.
for i,row in pmdata.iterrows():
    #folium.CircleMarker((row.latitude,row.longitude), radius=4, weight=1, color='blue', fill_color='blue', fill_opacity=.5, popup=(row.pm25)).add_to(newmap)
    folium.CircleMarker((row.latitude,row.longitude), radius=4, weight=1, color=colormap(pmdata.iloc[i]['pm25']), fill ='true', fill_opacity=.5, popup=(row.pm25)).add_to(newmap)

newmap.add_child(colormap)
#here we save an html version of the map - you can zoom in and out of it and interact with it!
#this .html file will be saved to your google drive folder. Download it to your computer and open it to interact with it!
newmap.save('newmap.html')
#you may need to refresh the page on your google drive folder to see the updated map!
#you may also need to close a few tabs so you don't run out of memory when opening the map :)


#note - you can also create a map for a subset of the total deployment time by calling the "thdatamod", "pmdatamod", or "no2datamod" variable above!


Now that we have our point map, showing us where pm2.5 values and potential hotspot locations are, let's do some clustering. This will allow us to see where multiple measurements exceed the threshold value, potentially indicating a local source of pollution or pollution transport.

In [ ]:
#Block 14
#let's start by setting 10 as the threshold value. Change this and see how the number of hotspots changes!
pmdata = pmdata.loc[(pmdata['pm25'] > 10)]

In [ ]:
#Block 15
#hierarchical clustering code

# bottom-up hierarchical clustering - agglomerative, not k-means, because number of clusters not defined before
hotspots = pmdata
coords = hotspots.loc[:,['latitude','longitude']].values

#preprocessing for hotspot clustering
#we have to convert to radians, because scikit-learn’s haversine metric needs radian units
kms_per_radian = 6371.0088

#epsilon is the max distance points can be from each other to count as a cluster
epsilon = 0.1 / kms_per_radian

#min_samples is the minimum cluster size for a hotspot to be formed, and here we also call the haversine metric
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))-(1 if -1 in set(cluster_labels) else 0)
outliers = coords[cluster_labels == -1]

#here is where we create the clusters after doing the background math above
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
outliers = coords[cluster_labels == -1]
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
#Block 16
#this portion of the code is going to tell us where the map should show up!

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)


centermost_points = clusters.map(get_centermost_point)
start_point=centermost_points[0]

#setting up the specifications for the map
hotspotmap = folium.Map(location= start_point, tiles='Stamen Terrain', zoom_start=14)
points=[]
#add a markers
for index, row in hotspots.iterrows():
    point=(row['latitude'] row['longitude'])
    if point not in points:
        new_point=(row['latitude'], row['longitude'])
        points.append(new_point)
for rep in centermost_points:
    folium.CircleMarker(location=rep, color='blue', fill=True, fill_color='blue',radius=15).add_to(hotspotmap)
for each in points:
    folium.CircleMarker(location=each, popup=(row.pm25), color='red', fill=True, fill_color='red',radius=7).add_to(hotspotmap)
    hotspotmap.add_child(folium.LatLngPopup())


#interactive html map showing hotspot clusters
hotspotmap.save('HotspotMap.html')

# Summary
Now that you have finished going through this notebook, you should be able to create time series graphs, basic maps, and perform clustering analysis techniques on hyperlocal environmental data. Additionally, you should have a basic knowledge of some of the different pollutants that can be measured to tell us more about the quality of our immediate environment.


## Reference Materials:

[NYCCAS Data](https://nyc-ehs.net/nyccas2020/web/report#Pollutant_Maps)

[United States EPA](https://www.epa.gov/)

[World Health Organization](https://www.who.int/)



## Python Library Documentation:

[Folium](https://python-visualization.github.io/folium/latest/)

[Pandas](https://pandas.pydata.org/)

[Matplotlib](https://matplotlib.org/)